In [10]:
import torch.optim as optim
import torch.nn as nn
import torch
from torchvision import models, datasets
import torchvision.transforms as transforms
import numpy as np
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [11]:
batch_size = 10
num_workers = 0

# define transformations
train_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# create the datasets
train_data = datasets.ImageFolder(root='data/train/train', 
    transform=train_transforms
)
test_data = datasets.ImageFolder(root='data/test/test', 
    transform=test_transforms
)
valid_data = datasets.ImageFolder(root='data/valid/valid', 
    transform=test_transforms
)
# create data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

loaders = {
    'train': train_loader,
    'test': test_loader,
    'valid': valid_loader
}

In [18]:
use_cuda = torch.cuda.is_available()
model = models.resnet50(pretrained=True)

for params in model.parameters():
    params.requires_grad = False

model.fc = nn.Sequential(
               nn.Linear(2048, 256),
               nn.ReLU(inplace=True),
               nn.Linear(256, 3))

model.fc.requires_grad = True

if use_cuda:
    model = model.cuda()

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} '.format(
            epoch, train_loss, valid_loss))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    return model


In [23]:
model = train(3, loaders, model, optimizer, criterion, use_cuda, 'resnet50.pt')

Epoch: 1 	Training Loss: 0.676889 	Validation Loss: 0.632400 
Validation loss decreased (inf --> 0.632400).  Saving model ...
Epoch: 2 	Training Loss: 0.674650 	Validation Loss: 0.594245 
Validation loss decreased (0.632400 --> 0.594245).  Saving model ...
Epoch: 3 	Training Loss: 0.666869 	Validation Loss: 0.613028 


In [22]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    preds_list = list()
    target_list = list()

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        
        # save the preds and targets for later analysis
        preds_list.append(output.data)
        target_list.append(target)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    
    return (preds_list, target_list)

# load the model that got the best validation accuracy
model.load_state_dict(torch.load('resnet50.pt'))


# call test function    
pr, tr = test(loaders, model, criterion, use_cuda)

Test Loss: 0.772279


Test Accuracy: 64% (389/600)


In [10]:
# cocnatenate into a sinfle array
pr_cat = torch.cat(pr, 0)
tr_cat = torch.cat(tr, 0)
 
# convert to numpy array
pr_cat = pr_cat.numpy()
tr_cat = tr_cat.numpy()

# convert scores to probs
pr_res = np.exp(pr_cat) / np.exp(pr_cat).sum(axis=1).reshape(-1, 1)

In [11]:
import pandas as pd
res_pd = pd.DataFrame(pr_res)

In [12]:
res_pd['target'] = tr_cat

In [15]:
(res_pd[[0,1,2]].idxmax(axis=1) == 2).sum()

19

In [19]:
(res_pd[[0,1,2]].idxmax(axis=1)  == res_pd['target']).sum()

402

In [20]:
 res_pd['target']

3